In [11]:
%config ZMQInteractiveShell.ast_node_interactivity='all'
%matplotlib inline
import warnings;warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
import os
import matplotlib.pyplot as plt

In [12]:
target = '标签'
uid = '申请编号'

############## Functions

def ana_dtypes(t):
    if t == 'str':
        cols = types[types == 'object'].index
    else:
        cols = types[types != 'object'].index
    # Save description
    cols_desc = d[cols].describe().T
    cols_desc.to_csv(f'./data/cols_desc_{t}.csv')
    # Return columns
    return cols

In [13]:
############## Load

d0 = pd.read_csv('./data/train_label.csv')
d0.shape
d0.head()

# Target Count
d0['标签'].value_counts()

(140000, 2)

,申请编号,标签
0,0,1
1,1,0
2,2,0
3,3,0
4,5,0


0    119000
1     21000
Name: 标签, dtype: int64

In [17]:
d1 = pd.read_csv('./data/train_Application.csv')
d1.shape
d1.head()
d1.dtypes.value_counts()

(140000, 31)

,申请编号,贷款类型,信用额度,贷款年金,商品价格,陪同申请人,出生日期距申请日期天数,工作日期距申请日期天数,注册日期距申请日期天数,身份认证日期距申请日期天数,...,申请人是否额外提供了文件6,申请人是否额外提供了文件7,申请人是否额外提供了文件8,申请人是否额外提供了文件9,贷款申请前1小时内征信查询次数,贷款申请前1天内征信查询次数,贷款申请前1周内征信查询次数,贷款申请前1个月内征信查询次数,贷款申请前1个季度内征信查询次数,贷款申请前1年内征信查询次数
0,0,0,460190.889355,17463.042019,419951.511045,7.0,-10107,-342,-5421,-3292,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,424370.659603,15585.046388,384597.975692,7.0,-13980,-1110,-3387,-826,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0
2,2,0,469330.587153,18837.558252,419951.511045,1.0,-13331,-2246,-3870,-171,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0
3,3,0,464188.353140,16754.587069,409850.500944,7.0,-16540,292204,-970,-2916,...,0,0,0,1,0.0,0.0,0.0,0.0,0.0,1.0
4,5,0,434196.988738,16165.009200,394698.985793,7.0,-17919,-11037,-9350,-3588,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,5.0


int64      20
float64    11
dtype: int64

In [20]:
d2 = pd.read_csv('./data/train_Personas.csv')
d2.shape
d2.dtypes.value_counts()
d2.head()

(140000, 32)

,申请编号,性别,是否有车,是否有房,孩子个数,客户收入,收入类型,教育程度,婚姻状态,居住状态,...,地址是否一致标志3,地址是否一致标志4,地址是否一致标志5,地址是否一致标志6,单位类型,社交圈违约信息2_2,社交圈违约信息2_1,社交圈违约信息1_2,社交圈违约信息1_1,最近一次换手机号码距申请日天数
0,0,1,0,0,0,237349.580536,1,2,3,1,...,0,0,0,0,5,0.0,0.0,0.0,0.0,-887.0
1,1,1,0,1,0,237236.363407,7,4,1,1,...,0,0,0,1,5,0.0,0.0,0.0,0.0,-271.0
2,2,0,1,1,-1,237321.276253,1,4,1,1,...,1,0,0,0,5,0.0,0.0,0.0,0.0,-332.0
3,3,0,0,1,-1,237304.293684,3,4,2,1,...,0,0,0,0,57,1.0,0.0,1.0,0.0,-204.0
4,5,0,0,0,-1,237321.276253,1,1,2,1,...,1,1,0,1,53,0.0,0.0,0.0,0.0,-1049.0


int64      18
float64    14
dtype: int64

In [29]:
### Merge Application

d11 = pd.merge(d0, d1, how='inner', on='申请编号')
d11.shape
d11.head()

d11 = pd.merge(d11, d2, how='inner', on='申请编号')
d11.shape
d11.head()

d11.to_csv(f'./tmp/0_merge11.csv')

(140000, 32)

,申请编号,标签,贷款类型,信用额度,贷款年金,商品价格,陪同申请人,出生日期距申请日期天数,工作日期距申请日期天数,注册日期距申请日期天数,...,申请人是否额外提供了文件6,申请人是否额外提供了文件7,申请人是否额外提供了文件8,申请人是否额外提供了文件9,贷款申请前1小时内征信查询次数,贷款申请前1天内征信查询次数,贷款申请前1周内征信查询次数,贷款申请前1个月内征信查询次数,贷款申请前1个季度内征信查询次数,贷款申请前1年内征信查询次数
0,0,1,0,460190.889355,17463.042019,419951.511045,7.0,-10107,-342,-5421,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,0,424370.659603,15585.046388,384597.975692,7.0,-13980,-1110,-3387,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0
2,2,0,0,469330.587153,18837.558252,419951.511045,1.0,-13331,-2246,-3870,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0
3,3,0,0,464188.353140,16754.587069,409850.500944,7.0,-16540,292204,-970,...,0,0,0,1,0.0,0.0,0.0,0.0,0.0,1.0
4,5,0,0,434196.988738,16165.009200,394698.985793,7.0,-17919,-11037,-9350,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,5.0


(140000, 63)

,申请编号,标签,贷款类型,信用额度,贷款年金,商品价格,陪同申请人,出生日期距申请日期天数,工作日期距申请日期天数,注册日期距申请日期天数,...,地址是否一致标志3,地址是否一致标志4,地址是否一致标志5,地址是否一致标志6,单位类型,社交圈违约信息2_2,社交圈违约信息2_1,社交圈违约信息1_2,社交圈违约信息1_1,最近一次换手机号码距申请日天数
0,0,1,0,460190.889355,17463.042019,419951.511045,7.0,-10107,-342,-5421,...,0,0,0,0,5,0.0,0.0,0.0,0.0,-887.0
1,1,0,0,424370.659603,15585.046388,384597.975692,7.0,-13980,-1110,-3387,...,0,0,0,1,5,0.0,0.0,0.0,0.0,-271.0
2,2,0,0,469330.587153,18837.558252,419951.511045,1.0,-13331,-2246,-3870,...,1,0,0,0,5,0.0,0.0,0.0,0.0,-332.0
3,3,0,0,464188.353140,16754.587069,409850.500944,7.0,-16540,292204,-970,...,0,0,0,0,57,1.0,0.0,1.0,0.0,-204.0
4,5,0,0,434196.988738,16165.009200,394698.985793,7.0,-17919,-11037,-9350,...,1,1,0,1,53,0.0,0.0,0.0,0.0,-1049.0


In [36]:
d11['标签'].value_counts()
d11.dtypes.value_counts()
# Description

cols_desc = d11.drop([uid, target], axis=1).describe().T
cols_desc.to_csv(f'./tmp/0_cols_desc.csv')

0    119000
1     21000
Name: 标签, dtype: int64

int64      38
float64    25
dtype: int64

In [38]:
d11.value_counts()

AttributeError: 'DataFrame' object has no attribute 'value_counts'

In [22]:
d3 = pd.read_csv('./data/train_History_Application.csv')
d3.shape
d3.dtypes.value_counts()
d3.head()

(647963, 32)

int64      19
float64    11
object      2
dtype: int64

,历史申请编号,申请编号,贷款类型,贷款年金,申请额度,信用额度,首付金额,商品价格,申请周内日,申请时点,...,商品类别,组合类型,产品类型,获客渠道,销售区域,行业,贷款期数,收益级别,产品组合,首付时间
0,1040733,173562,0,4444.0,106931,119633.0,NaN,106931.0,5,5,...,27,2,2,3,0,10,44.0,2,7.0,292204.0
1,450011,173562,0,3831.0,79211,90188.0,NaN,79211.0,7,4,...,27,2,2,0,4,10,44.0,3,7.0,292204.0
2,698695,173562,A,861.0,8327,4180.0,4367.0,8327.0,5,5,...,2,3,0,7,84,4,7.0,1,10.0,292204.0
3,924081,173562,A,1183.0,12568,12568.0,11.0,12568.0,7,4,...,7,3,0,7,83,5,14.0,3,12.0,292204.0
4,907875,173562,1,1990.0,11,39611.0,NaN,NaN,6,6,...,27,0,2,3,0,10,0.0,0,1.0,-560.0


In [23]:
types = d3.dtypes
types[types == 'object']

贷款类型           object
是否为最后一次申请记录    object
dtype: object

In [24]:
for col in '贷款类型', '是否为最后一次申请记录':
    d3[col].value_counts()

0    287881
A    284292
1     75624
Name: 贷款类型, dtype: int64

Y    644478
N      3485
Name: 是否为最后一次申请记录, dtype: int64

In [32]:
d4 = pd.read_csv('./data/train_History_Payment.csv')
d4.shape
d4.dtypes.value_counts()
d4.head()

(5257909, 8)

int64      5
float64    3
dtype: int64

,历史申请编号,申请编号,分期付款日期,有支付行为的分期付款,分期付款应支付时间,分期付款实际支付时间,本期贷款金额,本期还款金额
0,225186,81363,2,8,-934,-939.0,1233.0,1233.0
1,877876,81363,2,26,-426,-431.0,2000.0,2000.0
2,225186,81363,2,3,-1030,-1035.0,1233.0,1233.0
3,518289,81363,2,3,-182,-184.0,1308.0,1308.0
4,877876,81363,2,13,-690,-695.0,2000.0,2000.0
